In [13]:
from keras.models import load_model
import cv2
import numpy as np
from random import choice

In [14]:
model = load_model("model1.h5")

In [15]:
def mapper(val):
    return REV_CLASS_MAP[val]


def calculate_winner(move1, move2):
    if move1 == move2:
        return "Tie"

    if move1 == "rock":
        if move2 == "scissors":
            return "You"
        if move2 == "paper":
            return "Finn"

    if move1 == "paper":
        if move2 == "rock":
            return "You"
        if move2 == "scissors":
            return "Finn"

    if move1 == "scissors":
        if move2 == "paper":
            return "You"
        if move2 == "rock":
            return "Finn"


In [16]:
def finn_nr():
    finn = cv2.imread('images/Finn_neural.png', cv2.IMREAD_UNCHANGED)
    finn = cv2.resize(finn, (400, 400))
    finn_alpha = finn[:, :, 3]  # Extract the alpha channel

    frame_roi = frame[100:500, 0:400]  # Select the region of interest in the frame

    # Resize the alpha channel to match the size of the ROI
    finn_alpha_resized = cv2.resize(finn_alpha, (frame_roi.shape[1], frame_roi.shape[0]))

    # Create a three-channel alpha mask
    alpha_mask = np.zeros_like(frame_roi)
    for c in range(3):
        alpha_mask[:, :, c] = finn_alpha_resized

    # Apply the alpha mask to the ROI
    frame_roi_with_finn = cv2.bitwise_and(frame_roi, cv2.bitwise_not(alpha_mask))
    finn_with_background = cv2.bitwise_and(finn[:, :, :3], alpha_mask)
    combined = cv2.add(frame_roi_with_finn, finn_with_background)

    # Update the frame with the combined image
    frame[100:500, 0:400] = combined

def finn_sad():
    finn = cv2.imread('images/Finn_sad.png', cv2.IMREAD_UNCHANGED)
    finn = cv2.resize(finn, (400, 400))
    finn_alpha = finn[:, :, 3]  # Extract the alpha channel

    frame_roi = frame[100:500, 0:400]  # Select the region of interest in the frame

    # Resize the alpha channel to match the size of the ROI
    finn_alpha_resized = cv2.resize(finn_alpha, (frame_roi.shape[1], frame_roi.shape[0]))

    # Create a three-channel alpha mask
    alpha_mask = np.zeros_like(frame_roi)
    for c in range(3):
        alpha_mask[:, :, c] = finn_alpha_resized

    # Apply the alpha mask to the ROI
    frame_roi_with_finn = cv2.bitwise_and(frame_roi, cv2.bitwise_not(alpha_mask))
    finn_with_background = cv2.bitwise_and(finn[:, :, :3], alpha_mask)
    combined = cv2.add(frame_roi_with_finn, finn_with_background)

    # Update the frame with the combined image
    frame[100:500, 0:400] = combined

def finn_chanhr():
    finn = cv2.imread('images/Finn_chanhr.png', cv2.IMREAD_UNCHANGED)
    finn = cv2.resize(finn, (400, 400))
    finn_alpha = finn[:, :, 3]  # Extract the alpha channel

    frame_roi = frame[100:500, 0:400]  # Select the region of interest in the frame

    # Resize the alpha channel to match the size of the ROI
    finn_alpha_resized = cv2.resize(finn_alpha, (frame_roi.shape[1], frame_roi.shape[0]))

    # Create a three-channel alpha mask
    alpha_mask = np.zeros_like(frame_roi)
    for c in range(3):
        alpha_mask[:, :, c] = finn_alpha_resized

    # Apply the alpha mask to the ROI
    frame_roi_with_finn = cv2.bitwise_and(frame_roi, cv2.bitwise_not(alpha_mask))
    finn_with_background = cv2.bitwise_and(finn[:, :, :3], alpha_mask)
    combined = cv2.add(frame_roi_with_finn, finn_with_background)

    # Update the frame with the combined image
    frame[100:500, 0:400] = combined


In [18]:
import mediapipe as mp
import cv2
import numpy as np

# Load the trained model and label mapping
label_name_mapping = {'1': 'paper', '2': 'rock', '3': 'scissors', '4': 'nothing'}

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

# Initialize mediapipe hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

prev_move = None
computer_move_name = "" 
user_move_name = "" 
winner = ""
while True:
    ret, frame = cap.read()
    if not ret:
        continue
    frame = cv2.flip(frame, 1)

    # rectangle for user to play
    cv2.rectangle(frame, (250, 100), (650, 500), (255, 255, 255), 2)
    # rectangle for computer to play
    cv2.rectangle(frame, (800, 100), (1200, 500), (255, 255, 255), 2)

    # convert the frame to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Check if hand is within the specified rectangle
            hand_rect_w = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * image.shape[1]
            hand_rect_h = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * image.shape[0]

            if hand_rect_w > image.shape[1] - 500 and hand_rect_h < 500:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract hand landmarks coordinates
                count = 1
                coordinates = []
                for landmark in hand_landmarks.landmark:
                    if count == 1:
                        x0 = landmark.x
                        y0 = landmark.y
                        coordinates.extend([0, 0])
                        count += 1
                    else:
                        x = landmark.x
                        y = landmark.y
                        x = round(x - x0, 5)
                        y = round(y - y0, 5)
                        coordinates.extend([x, y])

                # Perform the prediction
                input_data = np.array([coordinates])
                predicted_probs = model.predict(input_data)[0]
                predicted_label = np.argmax(predicted_probs) + 1
                move_code = str(predicted_label)
                user_move_name = label_name_mapping[move_code]

                # predict the winner (human vs computer)
                if prev_move != user_move_name:
                    if user_move_name != "nothing":
                        computer_move_name = choice(['rock', 'paper', 'scissors'])
                        winner = calculate_winner(user_move_name, computer_move_name)
                    else:
                        computer_move_name = "nothing"
                        winner = "Waiting..."
                prev_move = user_move_name
                if computer_move_name != "none":

                    icon = cv2.imread(
                        "images/{}.png".format(computer_move_name))
                    icon = cv2.resize(icon, (400, 400))
                    frame[100:500, 250:650] = icon

                    if winner == 'You': finn_sad()
                    elif winner =='Finn': finn_chanhr()
                    else: finn_nr()



    
    # Display the information
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Finn's move: " + computer_move_name,
                            (250, 50), font, 1.2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Your move: " + user_move_name,
                            (800, 50), font, 1.2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Winner: " + winner,
                            (550, 600), font, 2, (0, 0, 255), 4, cv2.LINE_AA)

                
    cv2.imshow("Play game with Finn", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 24ms/step


2023-06-08 23:40:47.466 python[24226:464547] _TIPropertyValueIsValid called with 4 on nil context!
2023-06-08 23:40:47.466 python[24226:464547] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-06-08 23:40:47.466 python[24226:464547] Text input context does not respond to _valueForTIProperty:
2023-06-09 08:33:09.262 python[24226:464547] _TIPropertyValueIsValid called with 4 on nil context!
2023-06-09 08:33:09.262 python[24226:464547] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-06-09 08:33:09.263 python[24226:464547] Text input context does not respond to _valueForTIProperty:
2023-06-09 08:33:09.265 python[24226:464547] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_presentFunctionRowItemTextInputViewWithEndpoint:reply:) block performed very slowly (31930.03 secs).


: 

: 